In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import folium
import requests
import json
import os
import shutil

import requests
from bs4 import BeautifulSoup
from urllib.parse import urljoin
import re
import sys
import json
#!pip install googletrans
from googletrans import Translator

### Risk & Advice ( Travel_Advisory_API)

In [2]:
# This is a sample for one country
"""
url='https://www.reisewarnung.net/api?country=DE'
r = requests.get(url)
assert r.status_code==200,'API request failed'
print('Successful API request')
d=r.json()
print(json.dumps(d,indent=4))
"""

#Saving advisory of all in one file

url='https://www.reisewarnung.net/api'
r = requests.get(url)
assert r.status_code==200,'API request failed'
print('Successful API request')
d=r.json()['data']

info=[]
for i in d.keys():
    ccode=i
    cname=list(d[i]['lang']['de'].values())[0]
    crating=d[i]['situation']['rating']
    cadvice=d[i]['lang']['en']['advice']
    info.append([ccode,cname,crating,cadvice])
df=pd.DataFrame(info,columns=['Code','Country','Risk_Rating','Advice'])
df.to_csv('../data/info/advisory_info_from_api.csv',index=False)
print('Data is extraceted and writtern to a CSV file successfully')

Successful API request
Data is extraceted and writtern to a CSV file successfully


### Advisory info from countries

In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import requests
import folium
import json



from bs4 import BeautifulSoup
from urllib.parse import urljoin

import re
import sys

from nltk.corpus import words
from nltk.metrics.distance import (edit_distance,jaccard_distance)
from nltk.util import ngrams

#!pip install googletrans
#!pip install datapackage
#! pip install pyspellchecker
#!pip install pyenchant

from googletrans import Translator

translator = Translator()

#### Getting geojson data

In [5]:
with open('../json_csv_files/custom_world.geo.json') as f:
    d0=json.load(f)
    

with open('../json_csv_files/custom_world.geo.json') as f:
    d0=json.load(f)
    
print('No of countries : ',len(d0['features']))

clist0=[ i['properties']['name'] for i in d0['features'] ]

#print(sorted(clist0))

No of countries :  245


#### Getting filtered country name

In [8]:
df=pd.read_csv('../json_csv_files/modified_america_country_names.csv')
new_america_countries = list(df.iloc[:,0])

df=pd.read_csv('../json_csv_files/modified_austria_country_names.csv')
new_austria_countries = list(df.iloc[:,0])

df=pd.read_csv('../json_csv_files/modified_canada_country_names.csv')
new_canada_countries = list(df.iloc[:,0])

df=pd.read_csv('../json_csv_files/modified_australia_country_names.csv')
new_australia_countries = list(df.iloc[:,0])

### America data

In [51]:
url = "https://travel.state.gov/content/travel/en/traveladvisories/traveladvisories.html/"
base_site ='https://travel.state.gov'

r=requests.get(url)
print('Status : ',r.status_code)

html_content = r.content
soup=BeautifulSoup(html_content,'lxml')


with open('../data/usadvisory.html','wb') as f:
    f.write(soup.prettify('utf-8'))
    


all_links=soup.findAll('a', title=re.compile('Travel Advisory'),href=re.compile('(?i)travel-advisory'))
print('No of advisories available : ',len(all_links))

all_adv_links = [ urljoin(base_site, i.get('href')) for i in all_links]
all_countries = [ ' '.join(i.text.split( )[:-2]) for i in all_links]
print(all_adv_links[0])


#!mkdir -p ../data/Alldata

dir = "../data/Alldata"


if os.path.exists(dir):
    shutil.rmtree(dir)
os.makedirs(dir)

errlist1=[]

for i in range(len(new_america_countries)):
    url1=all_adv_links[i]
    r=requests.get(url1)
    #print('S:',r.status_code,end=' * ')
    print('',end='*')

    html_content = r.content
    soup1=BeautifulSoup(html_content,'lxml')
    
    text=soup1.find('div',{'class':'tsg-rwd-emergency-alert-text'}).text.replace('\n',' ').replace('\xa0','')
    text=str(text.encode(sys.stdout.encoding, errors='replace') )
    try:
        with open('../data/Alldata/'+new_america_countries[i]+'.txt','w') as f:
            f.write(text)
            f.write('\n'+'*'*100+'\n')
    except:
        #print('Error at index : ',i)
        errlist1.append(i)
        
print('Errorlist : ',errlist1)        
print('\nExtraction for all countries is done from american website')    

Status :  200
No of advisories available :  207
https://travel.state.gov/content/travel/en/traveladvisories/traveladvisories/Liechtenstein-Travel-Advisory.html
***************************************************************************************************************************************************************************************************************Errorlist :  []

Extraction for all countries is done from american website


### Canada

In [52]:
url = "https://travel.state.gov/content/travel/en/traveladvisories/traveladvisories.html/"
base_site ='https://travel.state.gov'

r=requests.get(url)
print('Status : ',r.status_code)


url = "https://travel.gc.ca/travelling/advisories"
base_site ='https://travel.gc.ca/'

r=requests.get(url)
print('Status : ',r.status_code)

html_content = r.content
soup=BeautifulSoup(html_content,'lxml')


with open('../data/canadaadvisory.html','wb') as f:
    f.write(soup.prettify('utf-8'))
    

all_links=soup.findAll('a',href=re.compile('destinations'))
print('No of advisories available : ',len(all_links))
    
    
all_adv_links = [ urljoin(base_site, i.get('href')) for i in all_links]
canada_countries = [ i.text for i in all_links]
print(all_adv_links[0])



errlist2=[]

for i in range(len(new_canada_countries)):
    url1=all_adv_links[i]
    try:
        r=requests.get(url1)
        print('',end='@')

        html_content = r.content
        soup1=BeautifulSoup(html_content,'lxml')
    
        text=soup1.find('details',{'id':'risk'}).text.replace('\n',' ')
        text = text.encode('ascii', 'ignore').decode('ascii')

    except:
        print('Error at request : ',i)
    #text=str(text.encode(sys.stdout.encoding, errors='replace') )
    try:
        with open('../data/Alldata/'+new_canada_countries[i]+'.txt','a') as f:
            f.write(text)
            f.write('\n'+'*'*100+'\n')
    except:
        #print('Error at index : ',i)
        errlist2.append(i)
        
print('Errorlist : ',errlist1)         
print('\nExtraction for all countries is done from canada website')  

Status :  200
Status :  200
No of advisories available :  230
https://travel.gc.ca/destinations/afghanistan
@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@Errorlist :  []

Extraction for all countries is done from canada website


### Austria

In [54]:
url = "https://www.bmeia.gv.at/reise-aufenthalt/reisewarnungen/"
base_site ='https://www.bmeia.gv.at/'

r=requests.get(url)
print('Status : ',r.status_code)


html_content = r.content
soup=BeautifulSoup(html_content,'lxml')


with open('../data/germanadvisory.html','wb') as f:
    f.write(soup.prettify('utf-8'))
    

all_links=soup.findAll('a',href=re.compile('reise-aufenthalt/reiseinformation/land'))
all_links=all_links[:207]  
print('No of advisories available : ',len(all_links))

all_adv_links = [ urljoin(base_site, i.get('href')) for i in all_links]
all_countries = [ i.text.strip() for i in all_links]
print(all_adv_links[0])


translator = Translator()
errlist3_1=[]
errlist3_2=[]

for i in range(0,len(all_countries)):
    url1=all_adv_links[i]
    try:
        r=requests.get(url1)
        print('',end='&')

        html_content = r.content
        soup1=BeautifulSoup(html_content,'lxml')
    
        ger_text = soup1.find('div',{'class':'panel-body'}).text
        eng_text = translator.translate(ger_text, src='de', dest='en')
        text     = eng_text.text.encode('ascii', 'ignore').decode('ascii')
    except:
        #print('Error at request/encoding :',i)
        errlist3_1.append(i)
    
    try:
        with open('../data/Alldata/'+new_austria_countries[i]+'.txt','a') as f:
            f.write(text)
            f.write('\n'+'+'*100+'\n')
    except:
        print('Error at index : ',i)
        errlist3_2.append(i)
        
print('Errorlist 3-1 : ',errlist3_1)         
print('Errorlist 3-2 : ',errlist3_2)         
print('Extraction for all countries is done from austria website')  

Status :  200
No of advisories available :  207
https://www.bmeia.gv.at/reise-aufenthalt/reiseinformation/land/afghanistan/
&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&Errorlist 3-1 :  [56]
Errorlist 3-2 :  []
Extraction for all countries is done from austria website
